In [1]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd
from sklearn import preprocessing


In [2]:
# загрузим данные в pandas DataFrame
df = pd.read_csv('C:/Users/Tom/DS/Jooble/data/train.tsv', sep='\t')

In [3]:
# посмотрим на полученный датафрейм
df.head()

,id_job,features
0,1864791934054678713,"2,9835,9999,9941,9945,9386,9899,9421,9954,9952..."
1,-7413918695841089440,"2,9082,9999,9700,9669,9981,9729,9822,9667,9526..."
2,-9223271545392256405,"2,9064,9999,9730,9585,9890,9740,9751,9538,9590..."
3,-9223240803898726824,"2,9402,9999,9711,9742,9975,9728,9924,9692,9514..."
4,-9223102057156184105,"2,9655,9996,9751,9722,9925,9736,9987,9733,9475..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 2 columns):
id_job      799 non-null int64
features    799 non-null object
dtypes: int64(1), object(1)
memory usage: 12.6+ KB


In [5]:
%%time
df_copy = df.copy()
df_copy = pd.concat([df_copy.iloc[:, 0], df_copy.iloc[:, 1].str.split(',', expand=True)], axis=1)
df_copy = df_copy.astype('int64')
df_copy.head()

Wall time: 136 ms


,id_job,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,256
0,1864791934054678713,2,9835,9999,9941,9945,9386,9899,9421,9954,...,8818,9954,9925,9934,8689,9958,9086,9114,9950,9875
1,-7413918695841089440,2,9082,9999,9700,9669,9981,9729,9822,9667,...,9979,9752,9695,9676,9974,9788,9955,9907,9747,9824
2,-9223271545392256405,2,9064,9999,9730,9585,9890,9740,9751,9538,...,9930,9705,9645,9652,9954,9582,9947,9876,9722,9791
3,-9223240803898726824,2,9402,9999,9711,9742,9975,9728,9924,9692,...,9959,9737,9611,9661,9912,9768,9963,9971,9767,9809
4,-9223102057156184105,2,9655,9996,9751,9722,9925,9736,9987,9733,...,9908,8208,9631,9571,9971,9407,9964,9990,9783,9907


In [6]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Columns: 258 entries, id_job to 256
dtypes: int64(258)
memory usage: 1.6 MB


In [7]:
# посмотрим есть ли в данных пропуски
df_copy.isnull().sum()

id_job    0
0         0
1         0
2         0
3         0
         ..
252       0
253       0
254       0
255       0
256       0
Length: 258, dtype: int64

## Выполнение тестового задания
Для стандартизации (нормализации) признаков удобно воспользоваться готовыми классами модуля [sklearn.preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html). В нашем случае воспользуемся классом [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) 

In [8]:
%%time
scaler = preprocessing.StandardScaler()
# scaler осуществляет z-score нормализацию каждого признака 
# при этом копирует (если не указано иное) значение среднего 
# и дисперсии этого признака для последующего использования
# X - объект numpy array (матрица стандартизированных признаков) 
X = scaler.fit_transform(df_copy.iloc[:, 2:])

Wall time: 9 ms


In [9]:
%%time
# метод idxmax возвращает индекс (имена столбцов, в нашем случае порядковые номера признаков)
# первого максимального элемента (если есть несколько равных значений) вдоль указанной оси
idx = df_copy.iloc[:, 2:].idxmax(axis=1)
idx

Wall time: 4 ms


0       19
1      162
2      162
3      162
4      125
      ... 
794      2
795    162
796    162
797    162
798    162
Length: 799, dtype: int64

In [10]:
# проверка
print('значение признака в столбце %d строки 0: %d' %(idx[0], df_copy.loc[0, idx[0]]))
print('значение максимального признака в строке 0: %d' %df_copy.iloc[0,2:].max())

значение признака в столбце 19 строки 0: 10000
значение максимального признака в строке 0: 10000


In [11]:
%%time
# найдем среднее значение признака с индексом idx по всей выборке
mean = df_copy.iloc[:, 2:].mean()
mean

Wall time: 1 ms


1      9528.702128
2      9867.720901
3      9746.367960
4      9687.679599
5      9766.359199
          ...     
252    9726.828536
253    9751.863579
254    9716.082603
255    9769.539424
256    9839.728411
Length: 256, dtype: float64

In [12]:
%%time
# абсолютное отклонение признака с индексом idx от его среднего значения mean 
abs_mean_diff = abs(df_copy.iloc[:, 2:].max(axis=1) - mean[idx].reset_index(drop=True))

Wall time: 54.1 ms


Результирующая таблица (в нашем случае это будет pandas DataFrame) должна содержать столбец id_job, а также стандартизированные признаки  то есть матриу X, также серии idx и abs_mean_diff. Создадим датафрейм result c помощью метода [concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) 

In [13]:
%%time
job_type = df_copy.iloc[0, 1]
#переименуем серии abs_mean_diff и idx в соответствии с требованием задания
abs_mean_diff.rename('max_feature_' + str(job_type) + '_abs_mean_diff', inplace=True)
idx.rename('max_feature_' + str(job_type) + '_index', inplace=True)
# создадим датафрейм data_scale со значениями стандартизированных признаков.
# столбцы подпишем в соответствии с требованием задания
data_scale = pd.DataFrame(X, columns=['feature_' + str(job_type) + '_stand_' + str({i}) for i in df_copy.columns[2:]])
result = pd.concat([df.iloc[:,0], data_scale, idx, abs_mean_diff], axis=1)
result

Wall time: 3 ms


,id_job,feature_2_stand_{1},feature_2_stand_{2},feature_2_stand_{3},feature_2_stand_{4},feature_2_stand_{5},feature_2_stand_{6},feature_2_stand_{7},feature_2_stand_{8},feature_2_stand_{9},...,feature_2_stand_{249},feature_2_stand_{250},feature_2_stand_{251},feature_2_stand_{252},feature_2_stand_{253},feature_2_stand_{254},feature_2_stand_{255},feature_2_stand_{256},max_feature_2_index,max_feature_2_abs_mean_diff
0,1864791934054678713,0.553928,0.234307,0.615962,0.624117,-1.230298,0.606144,-0.768501,0.638953,0.802290,...,0.715373,0.672200,-3.047238,0.735983,-1.631671,-1.426231,0.691022,0.212699,19,106.496871
1,-7413918695841089440,-0.807844,0.234307,-0.146743,-0.045306,0.694270,0.038556,0.189037,-0.110492,-0.464127,...,0.058535,0.040879,0.603478,0.194752,0.497777,0.452251,-0.086308,-0.094847,162,116.247810
2,-9223271545392256405,-0.840396,0.234307,-0.051801,-0.249044,0.399925,0.075283,0.019497,-0.447351,-0.273867,...,-0.084256,-0.017849,0.546657,-0.461092,0.478174,0.378817,-0.182039,-0.293848,162,116.247810
3,-9223240803898726824,-0.229136,0.234307,-0.111931,0.131751,0.674863,0.035218,0.432600,-0.045209,-0.499800,...,-0.181353,0.004174,0.427334,0.131078,0.517381,0.603856,-0.009724,-0.185302,162,116.247810
4,-9223102057156184105,0.228405,0.228952,0.014659,0.083242,0.513135,0.061928,0.583036,0.061854,-0.615740,...,-0.124237,-0.216054,0.594954,-1.018241,0.519832,0.648864,0.051543,0.405670,125,124.868586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,-9168651328982232304,0.704030,0.234307,0.524185,0.556205,-1.207656,0.415835,-0.264660,0.636342,0.406906,...,0.612564,0.429949,-2.606879,0.452633,-1.271454,-0.585297,0.637413,0.556428,2,131.279099
795,-9168503098892640218,0.611799,0.170054,0.685587,0.658074,-0.935952,0.766404,-0.288538,0.685957,0.754725,...,0.758210,0.681988,-0.672142,0.662758,-0.492208,-0.649255,0.779094,0.658943,162,112.247810
796,-9168200676135763139,0.747433,0.234307,0.723564,0.658074,-1.045928,0.733016,-0.049751,0.683346,0.722024,...,0.798192,0.757845,-0.442019,0.802841,-1.790951,-0.274980,0.798240,0.465973,162,116.247810
797,-9168200673481327350,0.666053,0.234307,0.707740,0.684754,-0.101433,0.826501,-0.278987,0.712070,0.840937,...,0.826750,0.723587,0.092093,0.783739,-0.195702,-0.066523,0.802070,0.701156,162,116.247810


In [14]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Columns: 259 entries, id_job to max_feature_2_abs_mean_diff
dtypes: float64(257), int64(2)
memory usage: 1.6 MB


## Использование иных (отличных от z-score) методов стандартизации. Пользовательские функции

In [15]:
# оформим последовательность действий для решения поставленной задачи в виде нескольких пользовательских функций
def great_primary_table(df):
    df_copy = df.copy()
    df_copy = pd.concat([df_copy.iloc[:, 0], df_copy.iloc[:, 1].str.split(',', expand=True)], axis=1)
    df_copy = df_copy.astype('int64')
    return df_copy


def abs_mean_diff(df):
    idx = df.idxmax(axis=1)
    mean = df.mean()
    result = abs(df.max(axis=1) - mean.loc[idx].reset_index(drop=True))
    return result, idx

# создадим словарь в котором ключами являются названия классов модуля sklearn.preprocessing a значениями - экземпляры классов
SCALE_FUNCTION = dict(
    [(attr, getattr(preprocessing, attr)) for attr in dir(preprocessing) if callable(getattr(preprocessing, attr))])

# следующая функция создает датафрейм с необходимыми по заданию столбцами (за исключением job_id).
# при этом можно указать какой метод стандартизации использовать (scale_func  - имя соответствующего класса модуля preprocessing)
# a также передать параметры этого класса (settings. Например для класса StandartScaler можно указать копировать ли значения 
# статистик mean и var)
def great_features_table(df, job_type, scale_func='StandardScaler', **settings):
    scaler = SCALE_FUNCTION[scale_func](**settings)
    data_scale = scaler.fit_transform(df)
    df_new = pd.DataFrame(data_scale,
                          columns=['feature_' + str(job_type) + '_stand_' + str({i}) for i in range(df.shape[1])])
    abs_mean, idx = abs_mean_diff(df)
    abs_mean.rename('max_feature_' + str(job_type) + '_abs_mean_diff', inplace=True)
    idx.rename('max_feature_' + str(job_type) + '_index', inplace=True)
    df_new = pd.concat([df_new, idx, abs_mean], axis=1)
    return df_new

In [16]:
%%time
# проверим работу пользовательских функций
df_primary = great_primary_table(df)
feature_table = great_features_table(df_primary.iloc[:, 2:], job_type)
result_test = pd.concat([df_primary.iloc[:, 0], feature_table], axis=1)
result_test

Wall time: 206 ms


,id_job,feature_2_stand_{0},feature_2_stand_{1},feature_2_stand_{2},feature_2_stand_{3},feature_2_stand_{4},feature_2_stand_{5},feature_2_stand_{6},feature_2_stand_{7},feature_2_stand_{8},...,feature_2_stand_{248},feature_2_stand_{249},feature_2_stand_{250},feature_2_stand_{251},feature_2_stand_{252},feature_2_stand_{253},feature_2_stand_{254},feature_2_stand_{255},max_feature_2_index,max_feature_2_abs_mean_diff
0,1864791934054678713,0.553928,0.234307,0.615962,0.624117,-1.230298,0.606144,-0.768501,0.638953,0.802290,...,0.715373,0.672200,-3.047238,0.735983,-1.631671,-1.426231,0.691022,0.212699,19,106.496871
1,-7413918695841089440,-0.807844,0.234307,-0.146743,-0.045306,0.694270,0.038556,0.189037,-0.110492,-0.464127,...,0.058535,0.040879,0.603478,0.194752,0.497777,0.452251,-0.086308,-0.094847,162,116.247810
2,-9223271545392256405,-0.840396,0.234307,-0.051801,-0.249044,0.399925,0.075283,0.019497,-0.447351,-0.273867,...,-0.084256,-0.017849,0.546657,-0.461092,0.478174,0.378817,-0.182039,-0.293848,162,116.247810
3,-9223240803898726824,-0.229136,0.234307,-0.111931,0.131751,0.674863,0.035218,0.432600,-0.045209,-0.499800,...,-0.181353,0.004174,0.427334,0.131078,0.517381,0.603856,-0.009724,-0.185302,162,116.247810
4,-9223102057156184105,0.228405,0.228952,0.014659,0.083242,0.513135,0.061928,0.583036,0.061854,-0.615740,...,-0.124237,-0.216054,0.594954,-1.018241,0.519832,0.648864,0.051543,0.405670,125,124.868586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,-9168651328982232304,0.704030,0.234307,0.524185,0.556205,-1.207656,0.415835,-0.264660,0.636342,0.406906,...,0.612564,0.429949,-2.606879,0.452633,-1.271454,-0.585297,0.637413,0.556428,2,131.279099
795,-9168503098892640218,0.611799,0.170054,0.685587,0.658074,-0.935952,0.766404,-0.288538,0.685957,0.754725,...,0.758210,0.681988,-0.672142,0.662758,-0.492208,-0.649255,0.779094,0.658943,162,112.247810
796,-9168200676135763139,0.747433,0.234307,0.723564,0.658074,-1.045928,0.733016,-0.049751,0.683346,0.722024,...,0.798192,0.757845,-0.442019,0.802841,-1.790951,-0.274980,0.798240,0.465973,162,116.247810
797,-9168200673481327350,0.666053,0.234307,0.707740,0.684754,-0.101433,0.826501,-0.278987,0.712070,0.840937,...,0.826750,0.723587,0.092093,0.783739,-0.195702,-0.066523,0.802070,0.701156,162,116.247810


## Анализ потребления памяти. 
Создадим датафрейм с рандомными значениями в котором 2 000 000 строк. Посмотрим сколько он занимает оперативной памяти.

In [17]:
df_memory_test = pd.DataFrame(np.random.random((2000000, 259)))
df_memory_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Columns: 259 entries, 0 to 258
dtypes: float64(259)
memory usage: 3.9 GB


In [23]:
# посмотрим сколько по времени займет расчет
%%time
feature_table_test = great_features_table(df_memory_test, job_type=2)   # пиковое потребелние ОЗУ - 14 GB

Wall time: 35.2 s


In [19]:
# освободим память
del df_memory_test

## Dask
Если остро стоит вопрос нехватки оперативной памяти для обработки данных, можно (и нужно) прибегнуть к библиотекам таким как PySpark, Hadoop, Dask.   
В нашем случае наиболее подходящим выбором будет [Dask](https://docs.dask.org/en/latest/) так как Dask копирует API Pandas, numpy, sklearn. Правда использование Dask оправдано только в случае если данные  не помещаются в ОЗУ. 

In [20]:
# импорт необходимых библиотек
import dask.dataframe as dd
import dask.array as da
import dask_ml.preprocessing as dask_preprocessing
#from dask.distributed import Client, LocalCluster

In [26]:
dask_df = dd.from_dask_array(da.random.random((2000000, 259)))
#dask_df = dask_df.persist() # сохранение dask DataFrame в ОЗУ (лучше использовать pandas df в таком случае)
dask_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258
npartitions=32,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
62500,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [27]:
dask_df.head()

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,256,257,258
0,0.566818,0.026248,0.472401,0.418659,0.936835,0.963836,0.607649,0.511670,0.283468,0.320446,...,0.069305,0.269524,0.195823,0.582063,0.590987,0.926587,0.540675,0.252320,0.813877,0.741773
1,0.152439,0.169249,0.772365,0.599871,0.214421,0.864775,0.033817,0.813980,0.896455,0.414139,...,0.618551,0.028618,0.457022,0.221837,0.904847,0.939056,0.692855,0.103561,0.427023,0.109599
2,0.342263,0.054568,0.597835,0.860036,0.133577,0.939086,0.588544,0.867791,0.752201,0.678753,...,0.547733,0.684300,0.251591,0.347403,0.400688,0.767395,0.266452,0.684436,0.941004,0.543676
3,0.214237,0.749329,0.629171,0.594857,0.736061,0.414109,0.908059,0.050235,0.224066,0.809523,...,0.432658,0.658091,0.563155,0.644714,0.894865,0.078659,0.665886,0.323754,0.015196,0.781663
4,0.832463,0.870075,0.662911,0.793182,0.096307,0.834182,0.800409,0.038097,0.590061,0.369169,...,0.138472,0.129939,0.157152,0.499193,0.820672,0.824901,0.641252,0.601013,0.323803,0.869677


### пользовательские функции адаптированные под Dask DataFrame

In [23]:
# оформим последовательность действий для решения поставленной задачи в виде нескольких пользовательских функций
def great_primary_dask_table(ddf):
    ddf_copy = ddf.copy()
    ddf_copy = dd.concat([ddf_copy.iloc[:, 0], ddf_copy.iloc[:, 1].str.split(',', expand=True)], axis=1)
    ddf_copy = ddf_copy.astype('int64')
    return ddf_copy


def abs_mean_diff_dask(ddf):
    idx = ddf.idxmax(axis=1)
    mean = ddf.mean().compute()
    result = ddf.max(axis=1) - mean.loc[idx].reset_index(drop=True)
    return result.abs(), idx

# создадим словарь в котором ключами являются названия классов модуля sklearn.preprocessing a значениями - экземпляры классов
DASK_SCALE_FUNCTION = dict(
    [(attr, getattr(dask_preprocessing, attr)) for attr in dir(dask_preprocessing) if callable(getattr(dask_preprocessing, attr))])

# следующая функция создает датафрейм с необходимыми по заданию столбцами (за исключением job_id).
# при этом можно указать какой метод стандартизации использовать (scale_func  - имя соответствующего класса модуля preprocessing)
# a также передать параметры этого класса (settings. Например для класса StandartScaler можно указать копировать ли значения 
# статистик mean и var)
def great_features_dask_table(ddf, job_type, scale_func='StandardScaler', **settings):
    abs_mean, idx = abs_mean_diff_dask(ddf)
    abs_mean.rename('max_feature_' + str(job_type) + '_abs_mean_diff', inplace=True)
    idx.rename('max_feature_' + str(job_type) + '_index', inplace=True)
    ddf = ddf.rename(columns=dict(zip(ddf.columns, ['feature_'+str(job_type)+'_stand_'+str({i}) for i in range(ddf.shape[1])])))
    scaler = DASK_SCALE_FUNCTION[scale_func](**settings)
    data_scale = scaler.fit_transform(ddf)                     
    df_new = dd.concat([data_scale, idx, abs_mean], axis=1)
    return df_new

## Немного диагностики

In [24]:
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler,visualize

In [28]:
with Profiler() as prof, ResourceProfiler(dt=0.5) as rprof, CacheProfiler() as cprof:
    dd.to_parquet(great_features_dask_table(dask_df, 2), 'C:/Users/Tom/DS/result.parq', engine='pyarrow')

In [34]:
from IPython.display import display
HTML(filename='C:/Users/Tom/DS/profile.html')

## Вывод
Как видно из графиков, максимальное потребление памяти - 3 gb. Таким образом можно обрабатывать практически любые данные. Главное - чтобы хватило места на жестком диске. 

In [30]:
# загрузим полученный в результате расчетов датафрейм из parquet файла
df2 = dd.read_parquet('C:/Users/Tom/DS/result.parq', engine='pyarrow')

In [31]:
df2.head()

,feature_2_stand_{0},feature_2_stand_{1},feature_2_stand_{2},feature_2_stand_{3},feature_2_stand_{4},feature_2_stand_{5},feature_2_stand_{6},feature_2_stand_{7},feature_2_stand_{8},feature_2_stand_{9},...,feature_2_stand_{251},feature_2_stand_{252},feature_2_stand_{253},feature_2_stand_{254},feature_2_stand_{255},feature_2_stand_{256},feature_2_stand_{257},feature_2_stand_{258},max_feature_2_index,max_feature_2_abs_mean_diff
index,,,,,,,,,,,,,,,,,,,,,
0,0.230587,-1.640289,-0.096080,-0.280992,1.512215,1.606353,0.372745,0.039063,-0.749029,-0.621679,...,-1.052369,0.283194,0.315823,1.476606,0.140249,-0.856819,1.087657,0.836738,80,0.498313
1,-1.205410,-1.144872,0.943114,0.346926,-0.989233,1.263210,-1.615328,1.086204,1.373567,-0.297053,...,-0.147899,-0.964344,1.403114,1.519787,0.667083,-1.372101,-0.252889,-1.353619,214,0.492660
2,-0.547590,-1.542177,0.338472,1.248425,-1.269165,1.520623,0.306555,1.272591,0.874059,0.619770,...,-0.859258,-0.529484,-0.343423,0.925343,-0.809095,0.639967,1.528185,0.150371,191,0.494024
3,-0.991251,0.864770,0.447034,0.329555,0.817012,-0.297877,1.413535,-1.559254,-0.954720,1.072857,...,0.219611,0.500168,1.368533,-1.459679,0.573720,-0.609383,-1.679976,0.974951,23,0.489707
4,1.151158,1.283085,0.563923,1.016769,-1.398217,1.157238,1.040575,-1.601297,0.312615,-0.452865,...,-1.186278,-0.003804,1.111509,1.124478,0.488439,0.350999,-0.610575,1.279901,191,0.489431
